# Autoencoder for MNIST compared to supervised model

### Imports and load the MNIST dataset

In [2]:
import tensorflow as tf

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train / 255.0
x_test = x_test / 255.0


2023-10-29 13:10:12.519316: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Vector size for latent variables and number of epochs

In [3]:
LATENT_SIZE = 32
EPOCHS = 10

### Define the autoencoder model consisting of an encoder and decoder

In [4]:
# Autoencoder using CNN layers
encoder = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),
    tf.keras.layers.Conv2D(16, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(LATENT_SIZE, activation="relu")
])

decoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(7 * 7 * 32, input_shape=(LATENT_SIZE,)),
    tf.keras.layers.Reshape((7, 7, 32)),
    tf.keras.layers.Conv2DTranspose(32, (3, 3), activation="relu", padding="same"),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2DTranspose(16, (3, 3), activation="relu", padding="same"),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2DTranspose(1, (3, 3), activation="sigmoid", padding="same"),
    tf.keras.layers.Reshape((28, 28))
])

img = tf.keras.layers.Input(shape = (28, 28))
latent_vector = encoder(img)
output = decoder(latent_vector)
autoencoder = tf.keras.models.Model(inputs = img, outputs = output)

### Compile and train the autoencoder

In [5]:
autoencoder.compile("nadam", loss = "binary_crossentropy")

autoencoder.fit(x_train, x_train, epochs=EPOCHS)

Epoch 1/10
1875/1875 [==============================] - 34s 17ms/step - loss: 0.1143
Epoch 2/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0865
Epoch 3/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0825
Epoch 4/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0809
Epoch 5/10
1875/1875 [==============================] - 31s 16ms/step - loss: 0.0800
Epoch 6/10
1875/1875 [==============================] - 31s 17ms/step - loss: 0.0793
Epoch 7/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0788
Epoch 8/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0784
Epoch 9/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0780
Epoch 10/10
1875/1875 [==============================] - 34s 18ms/step - loss: 0.0777


### Fine-tune the model with supervised learning with 10% of the training data

In [8]:
# Set up the model
model = tf.keras.models.Sequential([
    encoder,
    tf.keras.layers.Dense(10, activation="softmax")
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Use 1% of the labeled training examples for supervised learning
x_train_supervised = x_train[:600]
y_train_supervised = y_train[:600]

# Train the model
model.fit(
    x_train_supervised,
    y_train_supervised,
    epochs=EPOCHS,
)

# Evaluate the model on the test set
_, accuracy_autoencoder = model.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy after fine-tuning: {accuracy_autoencoder}")

Epoch 1/10
19/19 [==============================] - 1s 10ms/step - loss: 5.7505 - accuracy: 0.0233
Epoch 2/10
19/19 [==============================] - 0s 9ms/step - loss: 2.4571 - accuracy: 0.1483
Epoch 3/10
19/19 [==============================] - 0s 8ms/step - loss: 1.7202 - accuracy: 0.4317
Epoch 4/10
19/19 [==============================] - 0s 7ms/step - loss: 1.2827 - accuracy: 0.6167
Epoch 5/10
19/19 [==============================] - 0s 9ms/step - loss: 0.8721 - accuracy: 0.7883
Epoch 6/10
19/19 [==============================] - 0s 9ms/step - loss: 0.5985 - accuracy: 0.8500
Epoch 7/10
19/19 [==============================] - 0s 8ms/step - loss: 0.4603 - accuracy: 0.8717
Epoch 8/10
19/19 [==============================] - 0s 7ms/step - loss: 0.3751 - accuracy: 0.8917
Epoch 9/10
19/19 [==============================] - 0s 9ms/step - loss: 0.3037 - accuracy: 0.9233
Epoch 10/10
19/19 [==============================] - 0s 10ms/step - loss: 0.2631 - accuracy: 0.9250
Accuracy after fi

## Train a CNN model on 1% of the labeled data

In [9]:
# Set up the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Train the model
model.fit(
    x_train_supervised,
    y_train_supervised,
    epochs=EPOCHS,
)

# Evaluate the model on the test set
_, accuracy_cnn = model.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy after fine-tuning: {accuracy_cnn}")

Epoch 1/10
19/19 [==============================] - 1s 18ms/step - loss: 2.0274 - accuracy: 0.4100
Epoch 2/10
19/19 [==============================] - 0s 17ms/step - loss: 0.9708 - accuracy: 0.7600
Epoch 3/10
19/19 [==============================] - 0s 15ms/step - loss: 0.5264 - accuracy: 0.8417
Epoch 4/10
19/19 [==============================] - 0s 15ms/step - loss: 0.3506 - accuracy: 0.8900
Epoch 5/10
19/19 [==============================] - 0s 13ms/step - loss: 0.2720 - accuracy: 0.9183
Epoch 6/10
19/19 [==============================] - 0s 13ms/step - loss: 0.1842 - accuracy: 0.9417
Epoch 7/10
19/19 [==============================] - 0s 12ms/step - loss: 0.1399 - accuracy: 0.9617
Epoch 8/10
19/19 [==============================] - 0s 12ms/step - loss: 0.0896 - accuracy: 0.9767
Epoch 9/10
19/19 [==============================] - 0s 12ms/step - loss: 0.0658 - accuracy: 0.9917
Epoch 10/10
19/19 [==============================] - 0s 12ms/step - loss: 0.0429 - accuracy: 0.9933
Accuracy 

### Compare the performance of the autoencoder and CNN models

In [10]:
print(f'Unsupervised CNN autoencoder with supervised fine-tuning accuracy: {accuracy_autoencoder}')
print(f'Supervised CNN accuracy: {accuracy_cnn}')

Unsupervised CNN autoencoder with supervised fine-tuning accuracy: 0.868399977684021
Supervised CNN accuracy: 0.9175000190734863
